In [2]:
# !touch kaggle.json
# !echo '{"username":"shadanparvez","key":"e382a59baf026b6454dc73a90fd051ef"}'>> kaggle.json

# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# print('connection done!')

# !kaggle datasets download -d nexuswho/hinglish-top-dataset
# !unzip hinglish-top-dataset.zip

In [3]:
import pandas as pd
# df1= pd.read_csv("/content/Human Annotated Data/train.tsv",delimiter='\t')
# df2= pd.read_csv("/content/Human Annotated Data/test.tsv",delimiter='\t')
# df= pd.concat([df1, df2], ignore_index=True)
# df

df1 =  pd.read_csv("/content/Synthetically Generated Data/train.tsv",delimiter='\t')
df1 = df1[:50000]

In [4]:
len(df1)

50000

In [5]:
data=df1[['en_query', 'cs_query']]

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   en_query  50000 non-null  object
 1   cs_query  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [7]:
import re
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^\w\s:]', '', text)  # Keep only digits, colons, and alphabetic characters
    #text = re.sub(r'[^a-zA-Z]',' ', text)
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    return text

data['en_query']=data.en_query.apply(preprocess)
data['cs_query']=data.cs_query.apply(preprocess)

data[:5]

<ipython-input-7-ac5b1e49f1dc>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['en_query']=data.en_query.apply(preprocess)
<ipython-input-7-ac5b1e49f1dc>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cs_query']=data.cs_query.apply(preprocess)


,en_query,cs_query
0,who is the featured performer at the grand ole...,is saturday night the grand ole opry me featur...
1,remind me of my meeting at 12 am,mujhe 12 am ko meri meeting ke baare me yaad d...
2,set an alarm for 5 am,subha 5 baje ke liye ek alarm set karen
3,what s will weather be like at sydney at 2 pm,dopaher 2 baje sydney me mausam kaisa hoga
4,what is the weather in canada,canada me mausam kaisa hai


In [8]:
data['cs_query'] = data['cs_query'].apply(lambda x : 'START_ '+ x + ' _END')

<ipython-input-8-f4d266d6090d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cs_query'] = data['cs_query'].apply(lambda x : 'START_ '+ x + ' _END')


In [9]:
all_eng_words=set()
for eng in data['en_query']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hin_words=set()
for hin in data['cs_query']:
    for word in hin.split():
        if word not in all_hin_words:
            all_hin_words.add(word)


In [10]:
data['length_en']=data['en_query'].apply(lambda x:len(x.split(" ")))
data['length_cs']=data['cs_query'].apply(lambda x:len(x.split(" ")))

<ipython-input-10-072a9497302d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['length_en']=data['en_query'].apply(lambda x:len(x.split(" ")))


In [11]:
max_length_src=max(data.length_en) #31
max_length_tar=max(data.length_cs) #32
print(max(data.length_en),max(data.length_cs) )

50 55


In [12]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hin_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hin_words)
num_decoder_tokens+=1
num_encoder_tokens, num_decoder_tokens


(10395, 10995)

In [13]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])
#input_token_index

In [14]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [15]:
data

,en_query,cs_query,length_en,length_cs
0,who is the featured performer at the grand ole...,START_ is saturday night the grand ole opry me...,13,14
1,remind me of my meeting at 12 am,START_ mujhe 12 am ko meri meeting ke baare me...,8,13
2,set an alarm for 5 am,START_ subha 5 baje ke liye ek alarm set karen...,6,11
3,what s will weather be like at sydney at 2 pm,START_ dopaher 2 baje sydney me mausam kaisa h...,11,10
4,what is the weather in canada,START_ canada me mausam kaisa hai _END,7,8
...,...,...,...,...
49995,what time will we make to uncle bob house from...,START_ fayetteville se uncle bob ke ghar tak k...,11,14
49996,reset all alarms,START_ sabhi alarms ko reser karo _END,3,7
49997,set timer for every 10 min please,START_ please har 10 min ke liye timer set kar...,7,11
49998,program an alarm for tomorrow at 6,START_ kal 6 bajhe ke liye alarm set kare _END,7,10


In [16]:
from sklearn.model_selection import train_test_split
X, y = data['en_query'], data['cs_query']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((40000,), (10000,))

In [17]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep

                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [18]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense,TimeDistributed,Embedding,Bidirectional

latent_dim = 300
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens+1, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [19]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens+1, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [20]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.summary()
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 32
epochs = 100

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 300)            3118800   ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, None, 300)            3298800   ['input_2[0][0]']             
                                                                                              

In [21]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

#------------------------------------------------------------------------------
#                          checkpoint and earlystopping
#------------------------------------------------------------------------------

# checkpoint path
filepath="cnn-{epoch:02d}-{val_accuracy:.2f}.hdf5"

checkpoint = ModelCheckpoint(filepath,
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True,
                             mode='max')

earlystopping = EarlyStopping(monitor='val_accuracy',
                              patience=7,
                              restore_best_weights=True)

reduce_LR = ReduceLROnPlateau(monitor='val_accuracy',
                                                factor=.1,
                                                patience=5,
                                                verbose=1)
callback = [earlystopping, reduce_LR, checkpoint]

In [22]:
import numpy as np
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples/batch_size,
                    epochs=100,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples/batch_size,
                    callbacks=callback)

<ipython-input-22-957b3e8ed109>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),


Epoch 1/100
1250/1250 [==============================] - ETA: 0s - loss: 4.0515 - accuracy: 0.3401
Epoch 1: val_accuracy improved from -inf to 0.48210, saving model to cnn-01-0.48.hdf5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1250/1250 [==============================] - 226s 169ms/step - loss: 4.0515 - accuracy: 0.3401 - val_loss: 2.9259 - val_accuracy: 0.4821 - lr: 0.0010
Epoch 2/100
1250/1250 [==============================] - ETA: 0s - loss: 2.4291 - accuracy: 0.5437
Epoch 2: val_accuracy improved from 0.48210 to 0.58764, saving model to cnn-02-0.59.hdf5
1250/1250 [==============================] - 183s 146ms/step - loss: 2.4291 - accuracy: 0.5437 - val_loss: 2.2420 - val_accuracy: 0.5876 - lr: 0.0010
Epoch 3/100
1250/1250 [==============================] - ETA: 0s - loss: 1.8118 - accuracy: 0.6330
Epoch 3: val_accuracy improved from 0.58764 to 0.63826, saving model to cnn-03-0.64.hdf5
1250/1250 [==============================] - 190s 152ms/step - loss: 1.8118 - accuracy: 0.6330 - val_loss: 1.9484 - val_accuracy: 0.6383 - lr: 0.0010
Epoch 4/100
1250/1250 [==============================] - ETA: 0s - loss: 1.4097 - accuracy: 0.6927
Epoch 4: val_accuracy improved from 0.63826 to 0.66463, saving model to cnn

In [23]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [24]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [27]:
def convert_vector(input_text,max_length_src):
    # Preprocess the input text
    input_seq = np.zeros((1, max_length_src), dtype='float32')
    for t, word in enumerate(input_text.split()):
        input_seq[0, t] = input_token_index.get(word, 0)  # Use 0 for unknown words

    return input_seq

def predict_by_model(x):
    decoded_sentence = decode_sequence(convert_vector(x,31))
    return decoded_sentence[:-4]

df_val= pd.read_csv("/content/Human Annotated Data/validation.tsv",delimiter='\t')

res=df_val[['en_query', 'cs_query']]
res=res[:100]
res['en_query']=res.en_query.apply(preprocess)
res['gen_query']=res.en_query.apply(predict_by_model)


res

1/1 [==============================] - 0s 25ms/step


,en_query,cs_query,gen_query
0,pause my timer,mere timer ko roko,mere timer ko pause kare
1,please make video message for liz,Please Liz ke liye video message banaye,please queens ke liye ek video message bhejo
2,create an alarm at 4 pm and 9 pm on tuesday,Alarm create karo at 4 pm and 9 pm on tuesday,tuesday ko 9 pm aur 4 pm ko alarm create kare
3,library events in cedar park,Library events Cedar Park me hai,cedar park me library events
4,please pause timer,Please timer ko rokey,please timer ko rokey
...,...,...,...
95,skip this song,Is song ko chhod dijiye,ye song ko skip kare
96,can you remind my husband to buy our son a bik...,kya ap mere pati ko thursday ke din humare bet...,kya aap muje remind kara sakte he to attend my o
97,message jane asking if she can come over,Jane ko message karke pucho agar wah aa sakti hai,jane ko message karo aur pucho ki she was the...
98,how much time is left on my timer,mere timer me kitna waqt bacha hai,mere timer par kitna time bacha he


In [28]:
res.to_csv('result.csv', index=False)
print("done")

done
